In [1]:
# Import required packages
import gc
import os
import shutil

import numpy as np
import progressbar as pb
import sklearn.utils as sku

import Config as conf
import CSV as csv

In [2]:
# Import & shuffle CSV data
xx, yy = csv.getCSV()
xx, yy = sku.shuffle(xx, yy, random_state=0)
xx = xx[..., np.newaxis]
xo, yo = xx, yy

-- Input directory not found. Calculate merged CSVs...
Create window of try_8 ...
Processing X:  ./Dataset/csi_try_8_001.csv
Processing X:  ./Dataset/csi_try_8_002.csv
Processing X:  ./Dataset/csi_try_8_003.csv
Processing X:  ./Dataset/csi_try_8_004.csv
Processing X:  ./Dataset/csi_try_8_005.csv
Saving X of 200 windows: ./Input_WINDOW3760_COL90_TH85/csi_001.csv
Processing X:  ./Dataset/csi_try_8_006.csv
Processing X:  ./Dataset/csi_try_8_007.csv
Processing X:  ./Dataset/csi_try_8_008.csv
Saving X of 200 windows: ./Input_WINDOW3760_COL90_TH85/csi_002.csv
Processing X:  ./Dataset/csi_try_8_009.csv
Processing X:  ./Dataset/csi_try_8_010.csv
Processing X:  ./Dataset/csi_try_8_011.csv
Processing X:  ./Dataset/csi_try_8_012.csv
Saving X of 200 windows: ./Input_WINDOW3760_COL90_TH85/csi_003.csv
Processing X:  ./Dataset/csi_try_8_013.csv
Processing X:  ./Dataset/csi_try_8_014.csv
Processing X:  ./Dataset/csi_try_8_015.csv
Processing X:  ./Dataset/csi_try_8_016.csv
Saving X of 200 windows: ./In

Processing Y:  ./Dataset/action_try_10_024.csv
Processing Y:  ./Dataset/action_try_10_025.csv
Saving Y of 200 windows: ./Input_WINDOW3760_COL90_TH85/action_014.csv
Processing Y:  ./Dataset/action_try_10_026.csv
Processing Y:  ./Dataset/action_try_10_027.csv
Processing Y:  ./Dataset/action_try_10_028.csv
Processing Y:  ./Dataset/action_try_10_029.csv
Saving Y of 200 windows: ./Input_WINDOW3760_COL90_TH85/action_015.csv
Processing Y:  ./Dataset/action_try_10_030.csv
Processing Y:  ./Dataset/action_try_10_031.csv
Processing Y:  ./Dataset/action_try_10_032.csv
Processing Y:  ./Dataset/action_try_10_033.csv
Saving Y of 200 windows: ./Input_WINDOW3760_COL90_TH85/action_016.csv
Processing Y:  ./Dataset/action_try_10_034.csv
Processing Y:  ./Dataset/action_try_10_035.csv
Processing Y:  ./Dataset/action_try_10_036.csv
Processing Y:  ./Dataset/action_try_10_037.csv
Saving Y of 200 windows: ./Input_WINDOW3760_COL90_TH85/action_017.csv
Processing Y:  ./Dataset/action_try_10_038.csv
Processing Y:  

In [3]:
# Import Keras
from keras.applications.vgg16 import VGG16
import keras.callbacks as kc
import keras.layers as kl
import keras.models as km
import keras.optimizers as ko
import keras.utils as ku

Using TensorFlow backend.


In [4]:
# Setup Keras VGG16 Model
print("GC Collected:", gc.collect())
nadam = ko.Nadam(lr=conf.LEARNING_RATE)
vModel = VGG16(
    input_shape=(conf.WINDOW_SIZE, conf.N_COLUMNS, 1),
    classes=conf.USE_NOACTIVITY and conf.N_CLASSES or conf.N_VALID_CLASSES,
    weights=None,
    include_top=False
)
fltv = kl.Flatten()(vModel.output)
dense = kl.Dense(conf.USE_NOACTIVITY and conf.N_CLASSES or conf.N_VALID_CLASSES, activation='softmax')(fltv)
model = km.Model(inputs=vModel.input, outputs=dense)
model.compile(
    loss="categorical_crossentropy",
    optimizer=nadam,
    metrics=["accuracy"]
)
model.summary()

GC Collected: 0
Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3760, 90, 1)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 3760, 90, 64)      640       
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 3760, 90, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 1880, 45, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 1880, 45, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 1880, 45, 128)     147584    
______________________________________________________

In [5]:
# Check output directory and prepare tensorboard
outputDir = conf.OUTPUT_PATH.format(conf.LEARNING_RATE, conf.BATCH_SIZE,
                                    conf.N_FILTERS, "")
if os.path.exists(outputDir):
    shutil.rmtree(outputDir)
os.makedirs(outputDir)
logDir = conf.LOG_PATH.format(conf.LEARNING_RATE, conf.BATCH_SIZE,
                              conf.N_FILTERS, "")
if os.path.exists(logDir):
    shutil.rmtree(logDir)
os.makedirs(logDir)
tensorboard = kc.TensorBoard(
    log_dir=logDir,
    histogram_freq=0,
    batch_size=conf.BATCH_SIZE,
    write_graph=True,
    write_grads=True,
    write_images=True,
    update_freq=10)
print(
    "If you have tensorboard in this environment, you can type below to see the result in tensorboard:"
)
print("    tensorboard --logdir=" + logDir)
print("Keras checkpoints and final result will be saved in here:")
print("    " + outputDir)

If you have tensorboard in this environment, you can type below to see the result in tensorboard:
    tensorboard --logdir=./Log_LR0.002_BATCH16_GWRRATE80/
Keras checkpoints and final result will be saved in here:
    ./Output_LR0.002_BATCH16_GWRRATE80/


In [6]:
# Modify activity:non-activity ratio
conf.ANOA_RATIO = 4
if conf.USE_NOACTIVITY:
    xx = np.empty([0, conf.WINDOW_SIZE, conf.N_COLUMNS, 1], float)
    yy = np.empty([0, conf.N_CLASSES], float)
    with pb.ProgressBar(max_value=len(yo)) as bar:
        bar.update(0)
        for i in range(len(yo)):
            if yo[i, 0] == 0:
                yy = np.concatenate((yy, yo[i, :][np.newaxis, ...]), axis=0)
                xx = np.concatenate((xx, xo[i, :][np.newaxis, ...]), axis=0)
            bar.update(i)
    print("Found Activity Y:", len(yy))
    noacmx = int(len(yy) * conf.ANOA_RATIO)
    print("Will find", noacmx, "Non-Activity windows")
    noac = 0
    with pb.ProgressBar(max_value=noacmx) as bar:
        bar.update(0)
        for i in range(len(yo)):
            if noac > noacmx:
                break
            if not yo[i, 0] == 0:
                yy = np.concatenate((yy, yo[i, :][np.newaxis, ...]), axis=0)
                xx = np.concatenate((xx, xo[i, :][np.newaxis, ...]), axis=0)
                bar.update(noac)
                noac += 1
    print("Input Ratio Modified, Final Shape: [xx]", xx.shape, "[yy]", yy.shape)

100% (4158 of 4158) |####################| Elapsed Time: 0:00:21 Time:  0:00:21
N/A% (0 of 836) |                        | Elapsed Time: 0:00:00 ETA:  --:--:--

Found Activity Y: 209
Will find 836 Non-Activity windows


100% (836 of 836) |######################| Elapsed Time: 0:07:24 Time:  0:07:24


Input Ratio Modified, Final Shape: [xx] (1046, 3760, 90, 1) [yy] (1046, 2)


In [ ]:
# Run KFold
conf.KFOLD = 8
conf.BATCH_SIZE = 64
conf.CP_PERIOD = 1
conf.N_EPOCH = 256
xx, yy = sku.shuffle(xx, yy, random_state=0)
for i in range(conf.KFOLD):
    # Run Garbage Collector
    print("Grabage Collecting Time! =>", gc.collect())
    
    # Roll the data
    xx = np.roll(xx, int(len(xx) / conf.KFOLD), axis=0)
    yy = np.roll(yy, int(len(yy) / conf.KFOLD), axis=0)

    # Data separation
    xTrain = xx[int(len(xx) / conf.KFOLD):]
    yTrain = yy[int(len(yy) / conf.KFOLD):]
    xEval = xx[:int(len(xx) / conf.KFOLD)]
    yEval = yy[:int(len(yy) / conf.KFOLD)]

    if not conf.USE_NOACTIVITY:
        # Remove NoActivity from ys
        yTrain = yTrain[:, 1:]
        yEval = yEval[:, 1:]

        # If there exists only one action, convert Y to binary form
        if yEval.shape[1] == 1:
            yTrain = ku.to_categorical(yTrain)
            yEval = ku.to_categorical(yEval)

    # Setup Keras Checkpoint
    checkpoint = kc.ModelCheckpoint(
        outputDir + "Checkpoint_K" + str(i + 1) + "_EPOCH{epoch}_ACC{val_acc:.6f}.h5", period=conf.CP_PERIOD)

    # Fit model (learn)
    print(
        str(i + 1) + " th fitting started. Endpoint is " + str(conf.KFOLD) +
        " th.")

    model.fit(
        xTrain,
        yTrain,
        batch_size=conf.BATCH_SIZE,
        epochs=conf.N_EPOCH,
        verbose=1,
        callbacks=[tensorboard, checkpoint],
        validation_data=(xEval, yEval))  # , validation_freq=2)
print("Epoch completed!")

Grabage Collecting Time! => 10
1 th fitting started. Endpoint is 8 th.
Instructions for updating:
Use tf.cast instead.
Train on 916 samples, validate on 130 samples
Epoch 1/256
916/916 [==============================] - 751s 819ms/step - loss: 3.0214 - acc: 0.8024 - val_loss: 3.4716 - val_acc: 0.7846
Epoch 2/256
916/916 [==============================] - 737s 804ms/step - loss: 3.1849 - acc: 0.8024 - val_loss: 3.4716 - val_acc: 0.7846
Epoch 3/256
916/916 [==============================] - 747s 815ms/step - loss: 3.1849 - acc: 0.8024 - val_loss: 3.4716 - val_acc: 0.7846
Epoch 4/256
916/916 [==============================] - 796s 869ms/step - loss: 3.1849 - acc: 0.8024 - val_loss: 3.4716 - val_acc: 0.7846
Epoch 5/256
832/916 [==========================>...] - ETA: 1:14 - loss: 3.0996 - acc: 0.8077

In [ ]:
# Saving model
print("Saving model & model information...")
modelYML = model.to_yaml()
with open(outputDir + "model.yml", "w") as yml:
    yml.write(modelYML)
modelJSON = model.to_json()
with open(outputDir + "model.json", "w") as json:
    json.write(modelJSON)
model.save(outputDir + "model.h5")
print('Model saved! Congratulations! You finished all processes of ML!')

In [ ]:
# Finished